# hit-song-classifier

## Multiclass Logistic Regression

The first thing we'll do is define our imports and declare any constants.

In [2]:
# Define our imports
import numpy as np
import matplotlib.pyplot as plt

from pyspark.sql import DataFrame, SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel, ParamGridBuilder
from pyspark.mllib.evaluation import MulticlassMetrics
from sklearn.metrics import ConfusionMatrixDisplay


# Define our constants
TARGET_DATA = "../../data/spotify_enhanced_dataset.csv"
RANDOM_SEED = 0
TARGET_FEATURES = [
    "popularity",
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "duration_ms",
    "time_signature",
]

Next, we shall split our data in a 60-20-20 train_test_val split. Then we shall build our set of inputs X. We don't want to include the features `[song, artist, year]` since they are keys. We also should define our label `class` since that is our classification feature.

In [3]:
def feature_transformer(df: DataFrame) -> DataFrame:
    """Assembles feature vectors in dataframe

    Args:
        df (DataFrame): input

    Returns:
        DataFrame: feature-label dataframe
    """
    va = VectorAssembler(inputCols=TARGET_FEATURES, outputCol="features")
    va_df = va.transform(df)
    return va_df


def load_data(filename: str) -> DataFrame:
    """Load data from file

    Args:
        filename (str): filename

    Returns:
        DataFrame: pyspark dataframe
    """
    spark = SparkSession.builder.appName("hit-song-classifier-rfc").getOrCreate()
    return spark.read.csv(filename, header=True, inferSchema=True)


def split_data(df: DataFrame) -> tuple[DataFrame, DataFrame, DataFrame]:
    """Splits data to train, val, test splits

    Args:
        df (DataFrame): full dataframe

    Returns:
        DataFrame: _description_
    """
    train, val, test = df.randomSplit([0.6, 0.2, 0.2], seed=RANDOM_SEED)
    return train, val, test


df = feature_transformer(load_data(TARGET_DATA))
train_df, val_df, test_df = split_data(df)
print(f"Train: {train_df.count()}\nTest: {test_df.count()}\nVal: {val_df.count()}")

Train: 19516
Test: 6527
Val: 6466


## Hyperparameter search: training multiple estimators

Here we will train multiple estimators with different hyperparameter settings. For Logistic Regression, the hyperparameters we've chosen include `regParam` and `maxIter`.

In [4]:
def train_estimators(
    data: DataFrame,
    estimator_type: any,
    param_name: str,
    param_vals: list[int],
    **kwargs,
) -> list:
    """Trains estimators

    Args:
        data (DataFrame): dataframe
        estimator_type (RandomForestClassifier): estimator
        param_name (str): parameter name
        param_vals (list[int]): parameter values

    Returns:
        list: list of trained estimators
    """
    estimators = []
    for val in param_vals:
        estimator = estimator_type(**{param_name: val}, **kwargs)
        one_vs_rest = OneVsRest(classifier=estimator, **kwargs)
        fitted_models = one_vs_rest.fit(data)
        estimators.append(fitted_models)
        print(f"Training {fitted_models}")
    return estimators

### Hyperparameter search: `regParam`


In [5]:
reg_params = [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1]
lr_reg_params_list = train_estimators(
    train_df,
    LinearSVC,
    "regParam",
    reg_params,
    featuresCol="features",
    labelCol="class",
)

Training OneVsRestModel_cbd82b0270d8
Training OneVsRestModel_0181fdcb68fa
Training OneVsRestModel_98b47e292f4c
Training OneVsRestModel_25faaf1df0b8
Training OneVsRestModel_9d6fee349e6a
Training OneVsRestModel_4b289b7f715d


We want to plot estimator scores to pick the estimator with the best score.

In [6]:
def score_model(models: list, data: DataFrame) -> list:
    """Scores models

    Args:
        estimators (list): list of estimators
        data (DataFrame): dataframe

    Returns:
        list: list of scores
    """
    evaluator = MulticlassClassificationEvaluator(
        labelCol="class", predictionCol="prediction", metricName="f1"
    )

    predicted = [model.transform(data) for model in models]
    return [evaluator.evaluate(pred) for pred in predicted]


print(f"train: {score_model(lr_reg_params_list, train_df)}")
print(f"val: {score_model(lr_reg_params_list, val_df)}")
print(f"test: {score_model(lr_reg_params_list, test_df)}")

train: [0.13205719024040458, 0.10834325375427266, 0.13458061998800216, 0.13265793648967977, 0.13406577539315034, 0.1140876226590048]
val: [0.13232938863146218, 0.10801010393897965, 0.13633903933288202, 0.1319990686874698, 0.1294175021375268, 0.11989673105000291]
test: [0.12361248287922887, 0.09949107454945452, 0.1341895001922275, 0.119127130566931, 0.12365435978872268, 0.108097287546134]


### Hyperparameter search: `maxIter`

In [7]:
max_iter = [1, 5, 10, 20, 50, 100]
lr_max_iter_list = train_estimators(
    train_df,
    LinearSVC,
    "maxIter",
    max_iter,
    featuresCol="features",
    labelCol="class",
)

Training OneVsRestModel_d05aa3bf4af8
Training OneVsRestModel_ca4c1b09c9f4
Training OneVsRestModel_0b664e524d80
Training OneVsRestModel_64da9152f050
Training OneVsRestModel_a1ff800980ad
Training OneVsRestModel_19e3e3f3d38f


### Evaluate the F1 score of the hyperparameters for `regParam` and `maxIter`

In [8]:
print(f"train: {score_model(lr_max_iter_list, train_df)}")
print(f"val: {score_model(lr_max_iter_list, val_df)}")
print(f"test: {score_model(lr_max_iter_list, test_df)}")

train: [0.1046909795198871, 0.1614179850136512, 0.14078960300950707, 0.12013608008374099, 0.11281864940278472, 0.15241708775855142]
val: [0.1057394112077128, 0.15881782603266523, 0.1426199931676656, 0.1197671152148004, 0.10659861728202229, 0.1455276319994137]
test: [0.09944186257920123, 0.1470506542900242, 0.1331166541774986, 0.11262073626860643, 0.1080676603214549, 0.14597890130054283]


In [9]:
def plot_estimator_scores(models: list, param_name: str, param_vals: list[int]):
    """Plots the scores of a list of estimators"""

    train_score = score_model(models, train_df)
    val_score = score_model(models, val_df)
    test_score = score_model(models, test_df)

    # Get the best validation score and the index of the best estimator
    best_val_score = max(val_score)
    best_val_idx = val_score.index(best_val_score)

    # Plot metrics
    plt.figure()
    plt.plot(np.arange(len(train_score)), train_score, "go-", label="train")
    plt.plot(np.arange(len(val_score)), val_score, "ro-", label="val")
    plt.plot(np.arange(len(test_score)), test_score, "k:", label="test")
    # Place an X at the best validation score
    plt.scatter(best_val_idx, best_val_score, marker="x", color="r", s=200)

    # Additional formatting
    plt.ylim(0, 1.05)
    plt.xticks(ticks=np.arange(len(train_score)), labels=param_vals)
    plt.ylabel("score")
    plt.xlabel(param_name)
    plt.title(f"{models[0].__class__.__name__} score vs. {param_name}")

    # Output scores at max_depth of the best_validation_score
    plt.text(
        3.7 if len(train_score) < 6 else 4.7,
        0.3,
        f"train = {train_score[best_val_idx]:.3f}",
        c="g",
        ha="right",
    )
    plt.text(
        3.7 if len(train_score) < 6 else 4.7,
        0.2,
        f"validate = {best_val_score:.3f}",
        c="r",
        ha="right",
    )
    plt.text(
        3.7 if len(train_score) < 6 else 4.7,
        0.1,
        f"test = {test_score[best_val_idx]:.3f}",
        c="k",
        ha="right",
    )
    plt.legend()

In [ ]:
%%time
plot_estimator_scores(lr_reg_params_list, "regParam", reg_params)

In [ ]:
%%time
plot_estimator_scores(lr_max_iter_list, "maxIter", max_iter)

## Evaluation using Confusion Matrices

In [ ]:
# Declare models and model names
model_list = [lr_reg_params_list, lr_max_iter_list]
model_names = ["OneVsRest SVM Regularization", "OneVsRest SVM Max Iterations"]


def generate_confusion_matrix(model_list: list, val_df: DataFrame, test_df: DataFrame):
    """Generates a confusion matrix for each model in model_list"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    for i, model in enumerate(model_list):
        val_score = score_model(model, val_df)
        best_val_score = max(val_score)
        best_val_idx = val_score.index(best_val_score)

        # Generate confusion matrix
        best_model = (
            model[best_val_idx].transform(test_df).select("class", "prediction")
        )
        predictionAndLabels = best_model.rdd.map(
            lambda row: (float(row["prediction"]), float(row["class"]))
        )
        class_names = (
            best_model.select("class")
            .distinct()
            .sort("class")
            .rdd.map(lambda row: row[0])
            .collect()
        )
        metrics = MulticlassMetrics(predictionAndLabels)
        con_mtrx = metrics.confusionMatrix().toArray().astype(int)

        # Plot confusion matrix
        disp = ConfusionMatrixDisplay(
            confusion_matrix=con_mtrx, display_labels=class_names
        )
        disp.plot(ax=axes[i], colorbar=False)
        axes[i % 2].set_title(model_names[i % 2])

    # Add colorbar
    fig.colorbar(disp.im_, ax=axes.ravel().tolist())


generate_confusion_matrix(model_list, val_df, test_df)

## Putting it all together using Cross-Validation

Since we evaluated the hyperparameters separately above, we should create a Pipeline in which the best hyperparameters are determined grid-wise. This should automate what we did above.

In [ ]:
def cross_validator(train_data: DataFrame) -> CrossValidatorModel:
    """Performs cross validation"""

    # Define the estimator
    lr = LinearSVC(featuresCol="features",
        labelCol="class",)
    one_vs_rest = OneVsRest(classifier=lr,
                            featuresCol="features",
                            labelCol="class",)

    # Define the parameter grid
    param_grid = (
        ParamGridBuilder()
        .addGrid(lr.regParam, reg_params)
        .addGrid(lr.maxIter, max_iter)
        .build()
    )

    # Define the evaluator using F1 score
    evaluator = MulticlassClassificationEvaluator(
        labelCol="class", predictionCol="prediction", metricName="f1"
    )

    # Create the cross validator with 5 folds
    cv = CrossValidator(
        estimator=one_vs_rest, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3
    )
    cv_model = cv.fit(train_data)

    return cv_model


cv_model = cross_validator(train_df)

In [ ]:
def generate_cross_val_confusion_matrix(
    cv_model: CrossValidatorModel, test_df: DataFrame
):
    """Generates a confusion matrix for the best model from cross validation"""

    # Generate confusion matrix
    predictionAndLabels = (
        cv_model.transform(test_df)
        .select("class", "prediction")
        .rdd.map(lambda row: (float(row["prediction"]), float(row["class"])))
    )
    class_names = (
        test_df.select("class")
        .distinct()
        .sort("class")
        .rdd.map(lambda row: row[0])
        .collect()
    )
    metrics = MulticlassMetrics(predictionAndLabels)
    con_mtrx = metrics.confusionMatrix().toArray().astype(int)

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=con_mtrx, display_labels=class_names)
    disp.plot()
    plt.title("Cross Validation Confusion Matrix")


generate_cross_val_confusion_matrix(cv_model, test_df)

### Explaining best hyperparameters

Now we should explain the best hyperparameters for the cross-validation model.

In [ ]:
print(cv_model.bestModel.explainParam("regParam"))
print(cv_model.bestModel.explainParam("maxIter"))